In [2]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plot
import subprocess
import os

BCCID= ['md1/']
mydir='/net/jam-amaro-shared/bccgc4/Clustering/Gromos/clustering/'
workingdir=mydir
filedir='/net/jam-amaro-shared/bccgc4/Strided_Traj/'
j='joined_traj_md'

traj1=md.load([filedir+j+'1.nc', filedir+j+'2.nc', filedir+j+'3.nc', filedir+j+'4.nc', filedir+j+'5.nc'], top=filedir+'protein.h5')
print (traj1)

#alignment by alpha carbons
alphacarbon_indices=traj1.topology.select('name CA')
print(alphacarbon_indices)

traj1.superpose(traj1, 0, alphacarbon_indices)

#proteinv selection (trajectory.pdb)
#protein1selection=traj1.topology.select('protein')
#print(protein1selection)
#protein1=traj1.atom_slice(protein1selection)
#print(protein1)
#print(protein1.topology)
#protein1.save_pdb(workingdir+'trajectory.pdb')

<mdtraj.Trajectory with 450000 frames, 3324 atoms, 220 residues, and unitcells>
[   8   27   54   60   72   83   99  111  135  159  174  196  203  214
  230  244  259  275  297  318  335  342  353  363  370  380  391  415
  425  445  456  466  482  489  499  518  533  543  560  579  601  620
  642  656  663  685  704  720  731  750  761  771  788  802  821  837
  849  859  870  884  899  921  942  949  963  985  992 1002 1016 1023
 1030 1050 1067 1081 1095 1105 1125 1142 1163 1182 1201 1213 1227 1249
 1256 1275 1288 1299 1311 1321 1332 1361 1367 1388 1410 1420 1437 1449
 1466 1488 1498 1515 1536 1548 1559 1581 1602 1626 1636 1646 1660 1671
 1682 1704 1725 1739 1754 1781 1787 1808 1815 1839 1854 1866 1882 1901
 1923 1938 1948 1964 1974 1988 2010 2025 2031 2047 2058 2074 2081 2097
 2109 2119 2143 2169 2175 2186 2206 2226 2245 2266 2290 2301 2308 2324
 2345 2366 2389 2395 2406 2416 2430 2447 2461 2477 2491 2508 2515 2531
 2550 2566 2582 2589 2610 2617 2629 2648 2662 2669 2691 2706 2727 27

<mdtraj.Trajectory with 450000 frames, 3324 atoms, 220 residues, and unitcells at 0x7fc17c7b5e80>

In [ ]:
traj1.save_pdb(workingdir+'trajectory.pdb')
print("YAY")

In [ ]:
###trajectory.pdb formatting
get_ipython().system('cat '+mydir+'trajectory.pdb | grep -v CRYST1 > '+mydir+'temp.pdb')
get_ipython().system('mv -f '+mydir+'temp.pdb '+mydir+'trajectory.pdb')
#get_ipython().system('perl -pi -e "s/END/ENDMDL/g" trajectory.pdb')

ref_frame=traj1[0]
print(ref_frame)
ref_frame.save_pdb(workingdir+'ref_frame.pdb')

get_ipython().system(" cat "+mydir+"ref_frame.pdb | awk '{print $6}' | sort -n | uniq > "+mydir+"resid_ref_frame.dat")

print("Woooh")
# resid_activesite.dat formatting
f=open(mydir+'resid_ref_frame.dat','r')
lines=f.readlines()
f.close()
f=open(mydir+'resid_ref_frame.dat','w')
for line in lines:
    if (line [0:1]!='\n'):
        if "," not in line:
            f.write(line)
f.close()
print("Duck Tales")

get_ipython().system('cat '+mydir+'resid_ref_frame.dat | awk \'{print "cat /net/jam-amaro-shared/bccgc4/Clustering/Gromos/first_frame.pdb | awk STARTif ($6==" $1  ") print $0 END" }\' | sed "s/START/\'{/g" | sed "s/END/}\'/g"  | csh > '+mydir+'ref_frame_correct$| csh > '+mydir+'ref_frame_correct_residues.pdb')

get_ipython().system('cat '+mydir+'ref_frame_correct_residues.pdb  | awk \'{printf $2 " "}\' > '+mydir+'ref_frame_atoms_indices.dat')
get_ipython().system('cat '+mydir+'ref_frame_correct_residues.pdb | grep " CA " | awk \'{ if ( NR%15 == 0){ {printf "%4i", $2} {printf "\\n"} } else {printf "%4i ", $2} }\' > '+mydir+'ref_frame.ndx')
get_ipython().system('cat /net/jam-amaro-shared/bccgc4/Clustering/Gromos/first_frame.pdb | grep " CA " | awk \'{ if ( NR%15 == 0){ {printf "%4i", $2} {printf "\\n"} } else {printf "%4i ", $2} }\' > '+mydir+'alpha_carbons_indices.ndx')
get_ipython().system('cat '+mydir+'alpha_carbons_indices.ndx '+mydir+'ref_frame.ndx > '+mydir+'selections.ndx')

print("Woooh")
f=open(mydir+'ref_frame.ndx','r+')
lines=f.readlines()
f.seek(0)
f.write('[ ref_frame_CA ] \n')
for line in lines:
    f.write(line)
f.close()

print("David tenat's in this one")
f=open(mydir+'alpha_carbons_indices.ndx','r+')
lines=f.readlines()
f.seek(0)
f.write('[ C-alpha ] \n')
for line in lines:
    f.write(line)
f.close()

print("And Lin Manuel Miranda")
get_ipython().system('cat '+mydir+'alpha_carbons_indices.ndx '+mydir+'ref_frame.ndx > '+mydir+'selections.ndx')

When you finish the above step, get the first frame of each trajectory and name it first_frame.pdb in the working directory.